Download corpus, judgments, etc

In [1]:
from ltr import download
corpus='http://es-learn-to-rank.labs.o19s.com/blog.jsonl'
judgments='http://es-learn-to-rank.labs.o19s.com/osc_judgments.txt'

download([corpus, judgments], dest='data/');

data/blog.jsonl already exists
data/osc_judgments.txt already exists


Parse out OSC's blog into `articles`

In [2]:
import json

articles = []

with open('data/blog.jsonl') as f:
    for line in f:
        blog = json.loads(line)
        articles.append(blog)

articles[-7]

{'title': "Lets Stop Saying 'Cognitive Search'",
 'url': 'https://opensourceconnections.com/blog/2019/05/28/lets-stop-saying-cognitive-search/',
 'author': 'doug-turnbull',
 'content': ' I consume a lot of search materials: blogs, webinars, papers, and marketing collateral. There’s a consistent theme that crops up over the years: buzzwords! I understand why this happens, and it’s not all negative. We want to invite others outside our community into what we do. Heck I write my own marketing collateral, so I get the urge to jump on a buzzword bandwagon from time-to-time. \n\n That being said, I want to tell you a dirty little secret. Nobody really knows what ‘cognitive search’ means in any concrete sense. Sit two people down, and ask them what problem ‘cognitive search’ solves, and you’ll get two different answers. Most likely they imagine some kind of silver-bullet solution to a unique, painful search relevance problem they’re experiencing. Problems that require careful, deep thought wh

Instantiate an Elasticsearch client (as opposed to a `SolrClient`). Hello LTR can work with either 

In [3]:
from ltr.client import ElasticClient
client=ElasticClient()

Reindex from the corpus into the `blog` index. The JSON file at `docker/elasticsearch/<index_name>_settings.json` is loaded to configure the index.

In [4]:
from ltr.index import rebuild
rebuild(client, index='blog', doc_src=articles)

Created index blog [Status: 200]
Streaming Bulk index DONE blog [Status: 201]


A set of features that we've come up with that seems to work well for OSC's blog. Note here, these are Elasticsearch specific

In [5]:
client.reset_ltr(index='tmdb')

config = {
    "featureset": {
        "features": [
            {
                "name": "title_term_match",
                "params": ["keywords"],
                "template": {
                    "constant_score": {
                       "filter": {
                            "match": {
                                "title": "{{keywords}}"
                            }
                       },
                       "boost": 1.0
                    }
                }
            },
           {
                "name": "content_bm25",
                "params": ["keywords"],
                "template": {
                    "match": {
                       "content": {
                          "query": "{{keywords}}"
                        }
                    }
                }
            },
            {
                "name": "title_phrase_bm25",
                "params": ["keywords"],
                "template": {
                    "match_phrase": {
                       "title": "{{ keywords }}"
                    }
                }
            },
            {
                "name": "title_phrase_match",
                "params": ["keywords"],
                "template": {
                    "constant_score": {
                       "filter": {
                            "match_phrase": {
                                "title": "{{keywords}}"
                            }
                       },
                       "boost": 1.0
                    }
                }
            },
            
            {
                "name": "stepwise_post_date",
                "params": ["keywords"],
                "template": {
                  "function_score": {
                     "query": {
                        "match_all": {
                        }
                     },
                     "boost_mode": "replace",
                     "score_mode": "sum",
                     "functions": [
                        {
                            "filter": {
                                "range": {
                                    "post_date": {
                                        "gte": "now-180d"
                                    }
                                }
                            },
                            "weight": "100"               
                        },
                        {
                            "filter": {
                                "range": {
                                    "post_date": {
                                        "gte": "now-360d"
                                    }
                                }
                            },
                            "weight": "100"               
                        },
                          {
                            "filter": {
                                "range": {
                                    "post_date": {
                                        "gte": "now-90d"
                                    }
                                }
                            },
                            "weight": "100"               
                        }

                     ]
                  }
                }
            },
            {
                "name": "category_phrase_bm25",
                "params": ["keywords"],
                "template": {
                    "match_phrase": {
                       "categories": "{{ keywords }}"
                    }
                }
            },
            {
                "name": "excerpt_bm25",
                "params": ["keywords"],
                "template": {
                    "match": {
                       "excerpt": "{{ keywords }}"
                    }
                }
            },
            {
                "name": "excerpt_phrase_bm25",
                "params": ["keywords"],
                "template": {
                    "match_phrase": {
                       "excerpt": "{{ keywords }}"
                    }
                }
            },
        ]
    },
    "validation": {
      "index": "blog",
      "params": {
          "keywords": "rambo"
      }

   }
}

client.create_featureset(index='blog', name='test', ftr_config=config)

Removed Default LTR feature store [Status: 404]
{"error":{"root_cause":[{"type":"index_not_found_exception","reason":"no such index [.ltrstore]","resource.type":"index_or_alias","resource.id":".ltrstore","index_uuid":"_na_","index":".ltrstore"}],"type":"index_not_found_exception","reason":"no such index [.ltrstore]","resource.type":"index_or_alias","resource.id":".ltrstore","index_uuid":"_na_","index":".ltrstore"},"status":404}
Initialize Default LTR feature store [Status: 200]
Create test feature set [Status: 201]


With features loaded, transform the judgment list (`query,doc,label`) into a full training set with `query,doc,label,ftr1,ftr2,...` to prepare for training

In [6]:
from ltr.judgments import judgments_open
from ltr.log import FeatureLogger
from itertools import groupby

ftr_logger=FeatureLogger(client, index='blog', feature_set='test')
with judgments_open('data/osc_judgments.txt') as judgment_list:
    for qid, query_judgments in groupby(judgment_list, key=lambda j: j.qid):
        ftr_logger.log_for_qid(judgments=query_judgments, 
                               qid=qid,
                               keywords=judgment_list.keywords(qid))

Recognizing 47 queries...


Train using RankyMcRankFace with the training set, optimizing search for a specific metric (here `NDCG@10`). Note `ltr.train` has additional capabilities for performing k-fold cross validaiton to ensure the model isn't overfit to training data.

The model is stored in the search engine named `test` which can be referred to later for searching.

In [7]:
from ltr.ranklib import train
trainLog = train(client,
                 training_set=ftr_logger.logged,
                 metric2t='NDCG@10',
                 featureSet='test',
                 index='blog',
                 modelName='test')



/var/folders/7_/cvjz84n54vx7zv_pw3gmdqr00000gn/T/RankyMcRankFace.jar already exists
Running java -jar /var/folders/7_/cvjz84n54vx7zv_pw3gmdqr00000gn/T/RankyMcRankFace.jar -ranker 6 -shrinkage 0.1 -metric2t NDCG@10 -tree 50 -bag 1 -leaf 10 -frate 1.0 -srate 1.0 -train /var/folders/7_/cvjz84n54vx7zv_pw3gmdqr00000gn/T/training.txt -save data/test_model.txt 
Delete model test: 404
Created Model test [Status: 201]
Model saved


In [18]:
!java -jar /var/folders/7_/cvjz84n54vx7zv_pw3gmdqr00000gn/T/RankyMcRankFace.jar -ranker 6 -shrinkage 0.1 -metric2t NDCG@10 -tree 50 -bag 1 -leaf 10 -frate 1.0 -srate 1.0 -train /var/folders/7_/cvjz84n54vx7zv_pw3gmdqr00000gn/T/training.txt


Discard orig. features
Training data:	/var/folders/7_/cvjz84n54vx7zv_pw3gmdqr00000gn/T/training.txt
Feature vector representation: Dense.
Ranking method:	LambdaMART
Feature description file:	Unspecified. All features will be used.
Train metric:	NDCG@10
Test metric:	NDCG@10
Feature normalization: No

[+] LambdaMART's Parameters:
No. of trees: 50
No. of leaves: 10
No. of threshold candidates: 256
Min leaf support: 1
Learning rate: 0.1
Stop early: 100 rounds without performance gain on validation data

Reading feature file [/var/folders/7_/cvjz84n54vx7zv_pw3gmdqr00000gn/T/training.txt]... [Done.]            
(47 ranked lists, 477 entries read)
Initializing... [Done]
---------------------------------
Training starts...
---------------------------------
#iter   | NDCG@10-T | NDCG@10-V | 
---------------------------------
1       | 0.9393    | 
2       | 0.9367    | 
3       | 0.9367    | 
4       | 0.9367    | 
5       | 0.937     | 
6       | 0.9401    | 
7       | 0.9414    | 
8       | 

Search! Pass some configuration in (`blog_fields`) for display purposes.

In [ ]:
blog_fields = {
    'title': 'title',
    'display_fields': ['url', 'author', 'categories', 'post_date']
}

from ltr import search
search(client, "beer", modelName='test', 
       index='blog', fields=blog_fields)